In [1]:
import numpy as np
from mini_torch import Conv2DTranspose, Tensor, Linear, sigmoid, Module, Adam, Conv2D, relu, binary_cross_entropy

if __name__ == "__main__":
    import torch
    import torch.nn as nn
    
    print("=" * 60)
    print("Testing Conv2DTranspose: Mini-Torch vs PyTorch")
    print("=" * 60)
    
    # Set random seed
    np.random.seed(42)
    torch.manual_seed(42)
    
    # Parameters
    batch, in_c, H, W = 2, 3, 4, 4
    out_c = 8
    kernel_size = 3
    stride = 2
    padding = 3
    
    # Create input and target
    input_data = np.random.randn(batch, in_c, H, W).astype(np.float32)
    
    # ============================================
    # Mini-Torch
    # ============================================
    print("\n[Mini-Torch]")
    
    mini_conv_transpose = Conv2DTranspose(
        in_channels=in_c, 
        out_channels=out_c, 
        kernel_size=kernel_size, 
        stride=stride,
        padding=padding
    )
    
    x_mini = Tensor(input_data)
    out_mini = mini_conv_transpose(x_mini)
    
    print(f"Output shape: {out_mini.shape}")
    print(f"Output sample values:\n{out_mini.data[0, 0]}")
    
    # Compute loss and backward
    target_mini = Tensor(np.random.randn(*out_mini.shape).astype(np.float32))
    loss_mini = ((out_mini - target_mini).pow(2)).mean()
    print(f"\nMSE Loss: {loss_mini.data.item():.6f}")
    
    loss_mini.backward()
    
    print(f"\nGradients:")
    print(f"  Input grad norm: {np.linalg.norm(x_mini.grad):.6f}")
    print(f"  Weight grad norm: {np.linalg.norm(mini_conv_transpose.W.grad):.6f}")
    print(f"  Bias grad norm: {np.linalg.norm(mini_conv_transpose.b.grad):.6f}")
    
    # ============================================
    # PyTorch
    # ============================================
    print("\n" + "=" * 60)
    print("[PyTorch]")
    
    pytorch_conv_transpose = nn.ConvTranspose2d(
        in_channels=in_c,
        out_channels=out_c,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
        bias=True
    )
    
    # Copy weights from mini-torch
    with torch.no_grad():
        # Note: PyTorch ConvTranspose2d weights are (in_channels, out_channels, kH, kW)
        pytorch_conv_transpose.weight.copy_(torch.from_numpy(mini_conv_transpose.W.data))
        pytorch_conv_transpose.bias.copy_(torch.from_numpy(mini_conv_transpose.b.data))
    
    x_torch = torch.from_numpy(input_data).requires_grad_(True)
    out_torch = pytorch_conv_transpose(x_torch)
    
    print(f"Output shape: {out_torch.shape}")
    print(f"Output sample values:\n{out_torch[0, 0].detach().numpy()}")
    
    # Compute loss and backward
    target_torch = torch.from_numpy(target_mini.data)
    loss_torch = ((out_torch - target_torch) ** 2).mean()
    print(f"\nMSE Loss: {loss_torch.item():.6f}")
    
    loss_torch.backward()
    
    print(f"\nGradients:")
    print(f"  Input grad norm: {torch.norm(x_torch.grad).item():.6f}")
    print(f"  Weight grad norm: {torch.norm(pytorch_conv_transpose.weight.grad).item():.6f}")
    print(f"  Bias grad norm: {torch.norm(pytorch_conv_transpose.bias.grad).item():.6f}")
    
    # ============================================
    # Comparison
    # ============================================
    print("\n" + "=" * 60)
    print("[Comparison]")
    print("=" * 60)
    
    # Compare outputs
    output_diff = np.abs(out_mini.data - out_torch.detach().numpy())
    print(f"\nForward pass:")
    print(f"  Max absolute difference: {output_diff.max():.2e}")
    print(f"  Mean absolute difference: {output_diff.mean():.2e}")
    print(f"  ✓ PASS" if output_diff.max() < 1e-5 else f"  ✗ FAIL")
    
    # Compare input gradients
    input_grad_diff = np.abs(x_mini.grad - x_torch.grad.numpy())
    print(f"\nInput gradients:")
    print(f"  Max absolute difference: {input_grad_diff.max():.2e}")
    print(f"  Mean absolute difference: {input_grad_diff.mean():.2e}")
    print(f"  ✓ PASS" if input_grad_diff.max() < 1e-5 else f"  ✗ FAIL")
    
    # Compare weight gradients
    weight_grad_diff = np.abs(mini_conv_transpose.W.grad - pytorch_conv_transpose.weight.grad.numpy())
    print(f"\nWeight gradients:")
    print(f"  Max absolute difference: {weight_grad_diff.max():.2e}")
    print(f"  Mean absolute difference: {weight_grad_diff.mean():.2e}")
    print(f"  ✓ PASS" if weight_grad_diff.max() < 1e-5 else f"  ✗ FAIL")
    
    # Compare bias gradients
    bias_grad_diff = np.abs(mini_conv_transpose.b.grad - pytorch_conv_transpose.bias.grad.numpy())
    print(f"\nBias gradients:")
    print(f"  Max absolute difference: {bias_grad_diff.max():.2e}")
    print(f"  Mean absolute difference: {bias_grad_diff.mean():.2e}")
    print(f"  ✓ PASS" if bias_grad_diff.max() < 1e-5 else f"  ✗ FAIL")
    
    print("\n" + "=" * 60)
    all_pass = (output_diff.max() < 1e-5 and 
                input_grad_diff.max() < 1e-5 and 
                weight_grad_diff.max() < 1e-5 and 
                bias_grad_diff.max() < 1e-5)
    print(f"Overall: {'✓ ALL TESTS PASSED' if all_pass else '✗ SOME TESTS FAILED'}")
    print("=" * 60)

Testing Conv2DTranspose: Mini-Torch vs PyTorch

[Mini-Torch]
Output shape: (2, 8, 3, 3)
Output sample values:
[[ 0.7351145  -0.24452999 -0.17586286]
 [ 0.3591942  -0.840743    0.11334914]
 [-0.26731247  0.19048168  0.24044836]]

MSE Loss: 1.261473

Gradients:
  Input grad norm: 0.187378
  Weight grad norm: 0.454128
  Bias grad norm: 0.165461

[PyTorch]
Output shape: torch.Size([2, 8, 3, 3])
Output sample values:
[[ 0.7351145  -0.24452999 -0.17586286]
 [ 0.3591942  -0.840743    0.11334914]
 [-0.26731247  0.19048168  0.24044836]]

MSE Loss: 1.261473

Gradients:
  Input grad norm: 0.187378
  Weight grad norm: 0.454128
  Bias grad norm: 0.165461

[Comparison]

Forward pass:
  Max absolute difference: 1.19e-07
  Mean absolute difference: 2.59e-09
  ✓ PASS

Input gradients:
  Max absolute difference: 7.45e-09
  Mean absolute difference: 4.83e-10
  ✓ PASS

Weight gradients:
  Max absolute difference: 7.45e-09
  Mean absolute difference: 6.75e-10
  ✓ PASS

Bias gradients:
  Max absolute differ